Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [3]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [7]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab3

In [8]:
median = np.median(DIST_MATRIX.reshape(1, -1))
ic(median)
# DIST_MATRIX[DIST_MATRIX > median] = np.inf
G = nx.Graph()
for c1, c2 in combinations(CITIES.itertuples(), 2):
    G.add_node(c1)
    G.add_node(c2)
    if DIST_MATRIX[c1.Index, c2.Index] <= median:
        G.add_edge(c1, c2)
nx.is_connected(G)

ic| median: np.float64(367.0694334013403)


True

## Greedy algorithm

In [9]:
from itertools import combinations
import random

# Assuming CITIES and DIST_MATRIX have been initialized as per your setup

def greedy_best_first_search(dist_matrix, start_index, end_index):
    current_index = start_index
    path = [current_index]
    total_distance = 0
    
    while current_index != end_index:
        # Find the closest unvisited city to the destination
        min_distance = float('inf')
        next_city = None
        for i in range(len(dist_matrix)):
            if i not in path and dist_matrix[current_index, i] < min_distance and i != current_index:
                min_distance = dist_matrix[current_index, i]
                next_city = i
        
        # If no next city is found (in case of isolated nodes), break
        if next_city is None:
            break
        
        path.append(next_city)
        total_distance += min_distance
        current_index = next_city
    
    return path, total_distance

# Select random start and end cities
num_cities = len(CITIES)
start_city_index = random.randint(0, num_cities - 1)
end_city_index = random.randint(0, num_cities - 1)

# Run the Greedy Best-First Search
path, total_distance = greedy_best_first_search(DIST_MATRIX, start_city_index, end_city_index)

# Display the results
start_city = CITIES.iloc[start_city_index]['name']
end_city = CITIES.iloc[end_city_index]['name']
print(f"Greedy path from {start_city} to {end_city} is {total_distance:.2f} km")
print("Path:", " -> ".join(CITIES.iloc[i]['name'] for i in path))


Greedy path from Syracuse to Pescara is 3189.61 km
Path: Syracuse -> Catania -> Reggio di Calabria -> Messina -> Palermo -> Salerno -> Naples -> Giugliano in Campania -> Latina -> Rome -> Terni -> Perugia -> Ancona -> Rimini -> Forlì -> Ravenna -> Ferrara -> Bologna -> Modena -> Reggio nell'Emilia -> Parma -> Piacenza -> Milan -> Monza -> Bergamo -> Brescia -> Verona -> Vicenza -> Padua -> Venice -> Trieste -> Bolzano -> Trento -> Novara -> Turin -> Genoa -> Leghorn -> Prato -> Florence -> Pescara
